In [3]:
# 导入库
import numpy as np

In [4]:
# 从文本读取数据，并划分
def get_data():
    train_data = np.genfromtxt('iris_training.csv', dtype='float', delimiter=',', skip_header=1)
    test_data = np.genfromtxt('iris_test.csv', dtype = 'float', delimiter=',', skip_header=1)

    # 读取的时候都是以float的形式读取的，但是最后一列是label，所以需要转换成int
    train_x = train_data[:, :-1]
    train_y = train_data[:, -1].astype(np.int64)
    
    test_x = test_data[:, :-1]
    test_y = test_data[:, -1].astype(np.int64)

    return train_x, train_y, test_x, test_y


In [5]:
# 获取划分的数据
train_x, train_y, test_x, test_y = get_data()

In [6]:
# 计算损失函数与梯度
def compute_softmax_loss(W, X, y, reg):
    
    # 计算softmax损失函数
    num_train = X.shape[0]
    scores = X.dot(W)
    scores -= np.max(scores, axis=1, keepdims=True)
    exp_scores = np.exp(scores)
    sum_exp_scores = np.sum(exp_scores, axis=1, keepdims=True)
    probs = exp_scores / sum_exp_scores

    # 计算损失函数
    correct_logprobs = -np.log(probs[np.arange(num_train), y])
    data_loss = np.sum(correct_logprobs) / num_train

    loss = data_loss + 0.5 * reg * np.sum(W * W)

    # 计算梯度
    dscores = probs.copy()
    dscores[np.arange(num_train), y] -= 1
    dscores /= num_train
    dW = X.T.dot(dscores)
    dW += reg * W

    return loss, dW

In [7]:
# 计算准确率
def acc(y, y_pred):
    return np.mean(y==y_pred)

In [8]:
# 预测函数
def predict(W, X):
    score = np.dot(X, W)
    y_pred = np.argmax(score, axis=1)
    
    return y_pred

In [11]:
# 训练预测模型
def train(X,y, X_test, y_test, learning_rate=1e-3, reg=1e-5, epochs=100, batch_size=200):
    num_train, dim = X.shape
    num_classes = np.max(y) + 1

    iterations_per_epoch = int(max(num_train / batch_size, 1))

    # 初始化权重
    W = 0.001 * np.random.randn(dim, num_classes)

    # 训练模型
    for epoch in range(epochs):
        # 随机打乱训练数据
        idx = np.random.permutation(num_train)
        
        for i in range(iterations_per_epoch):
            # 取出一个小批量的数据
            batch_idx = idx[i*batch_size:(i+1)*batch_size]
            X_batch = X[batch_idx]
            y_batch = y[batch_idx]

            # 计算损失和梯度
            loss, grad = compute_softmax_loss(W, X_batch, y_batch, reg)

            # 更新权重
            W += -learning_rate * grad

        # 计算训练集和测试集的准确率
        if epoch % 10 == 0:
            train_acc = acc(y_batch, predict(W, X_batch))
            test_acc = acc(y_test, predict(W, X_test))

            print('Epoch %d: train_acc = %f, test_acc = %f' % (epoch, train_acc, test_acc))


    return W

In [13]:
# 测试
max_epochs = 200
batch_size = 20
learning_rate = 0.1
reg = 0.01

# get training and testing data
train_x, train_y, test_x, test_y = get_data()
W = train(train_x, train_y, test_x, test_y, learning_rate, reg, max_epochs, batch_size)

# Classify two new flower samples.
def new_samples():
    return np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
new_x = new_samples()
predictions = predict(W, new_x)

print("New Samples, Class Predictions:    {}\n".format(predictions))


Epoch 0: train_acc = 0.200000, test_acc = 0.300000
Epoch 10: train_acc = 0.950000, test_acc = 0.966667
Epoch 20: train_acc = 0.750000, test_acc = 0.700000
Epoch 30: train_acc = 0.800000, test_acc = 0.833333
Epoch 40: train_acc = 0.900000, test_acc = 0.966667
Epoch 50: train_acc = 1.000000, test_acc = 0.966667
Epoch 60: train_acc = 0.950000, test_acc = 0.933333
Epoch 70: train_acc = 1.000000, test_acc = 1.000000
Epoch 80: train_acc = 0.850000, test_acc = 0.933333
Epoch 90: train_acc = 0.950000, test_acc = 0.933333
Epoch 100: train_acc = 1.000000, test_acc = 0.933333
Epoch 110: train_acc = 0.850000, test_acc = 0.800000
Epoch 120: train_acc = 1.000000, test_acc = 0.933333
Epoch 130: train_acc = 0.800000, test_acc = 0.933333
Epoch 140: train_acc = 1.000000, test_acc = 0.966667
Epoch 150: train_acc = 0.900000, test_acc = 0.966667
Epoch 160: train_acc = 1.000000, test_acc = 0.933333
Epoch 170: train_acc = 0.950000, test_acc = 0.933333
Epoch 180: train_acc = 0.950000, test_acc = 0.933333
Epoc